In [1]:
%cd ..

/Users/missoni/GitHub/livewire


In this notebook, we shall reproduce Livewire algorithm and compute the shortest path between two clicks.

# Input
read a DICOM slice image from file, convert it to a numpy array (http://www.osirix-viewer.com/datasets/)

In [2]:
import skimage

In [3]:
import dicom

In [4]:
df = dicom.read_file('IM-0001-0011.dcm')

In [5]:
image = skimage.img_as_uint(df.pixel_array)

In [6]:
print image, image[int(image.shape[0]*0.5)]

[[1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]
 ..., 
 [1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]] [1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1650 1641 1615 1633 1619 1624 1618 1616 1629 1620 1620
 1618 1630 1612 1636 1628 1648 1629 1642 1641 1636 1636 1603 1662 1596 1745
 1811 1860 1996 2214 2339 2216 1915 1680 1664 1770 1828 1784 1739 1755 1778
 1746 1684 1660 1671 1663 1633 1640 1722 1821 1860 1828 1796 1794 1796 1762
 1715 1688 1691 1686 1663 1643 1643 1652 1647 1628 1619 1640 1671 1680 1653
 1616 1603 1625 1654 1678 1704 1747 1792 1798 1751 1677 1637 1640 1656 1656
 164

# Compute gradient image 
(http://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.sobel)

In [7]:
from skimage import filters

In [8]:
edges = filters.sobel(image)*1000

# Display image

In [9]:
from skimage import io
import pylab
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


# Convert gradient image into a graph

In [10]:
from math import fabs

G = {}
edges.shape
rows, cols = edges.shape
for col in range(cols):
    for row in range(rows):
        
        neighbors = []
        if row > 0:
            neighbors.append( (row-1, col) )
        
        if row < rows-1:
            neighbors.append( (row+1, col) )
            
        if col > 0:
            neighbors.append( (row, col-1) )
        
        if col < cols-1:
            neighbors.append( (row, col+1) )
        
        dist = {}
        for n in neighbors:
            n[0], n[1]
            dist[n] = fabs(edges[row][col] - edges[n[0], n[1]])
            
        G[(row,col)] = dist

Re-use Dijkstra algorithm (http://code.activestate.com/recipes/119466-dijkstras-algorithm-for-shortest-paths/)

In [12]:
from dijkstra import shortestPath, shortestPath2

In [13]:
shortestPath2(G, (257, 315), (232, 310))

[(257, 315),
 (256, 315),
 (255, 315),
 (254, 315),
 (254, 314),
 (253, 314),
 (253, 313),
 (252, 313),
 (251, 313),
 (251, 312),
 (250, 312),
 (249, 312),
 (249, 311),
 (248, 311),
 (248, 310),
 (247, 310),
 (246, 310),
 (245, 310),
 (244, 310),
 (243, 310),
 (242, 310),
 (241, 310),
 (240, 310),
 (239, 310),
 (238, 310),
 (237, 310),
 (236, 310),
 (235, 310),
 (234, 310),
 (233, 310),
 (232, 310)]

In [14]:
start_point = None
print start_point

def button_pressed(event):
    global start_point
    
    print event.xdata, event.ydata
    if start_point is None:
        start_point = (int(event.xdata), int(event.ydata))
        print 'first click'
        
    else:
        print 'second click'
        path = shortestPath2(G, start_point, (int(event.xdata), int(event.ydata)))
        print path
        start_point = None
        plt.plot(np.array(path)[:,0], np.array(path)[:,1])

import numpy as np
io.imshow(edges, cmap=plt.cm.gray )
plt.connect('button_release_event', button_pressed)
io.show()

None


/Library/Python/2.7/site-packages/skimage/io/_plugins/matplotlib_plugin.py:75: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  warnings.warn("Float image out of standard range; displaying image "


289.1 116.833333333
first click
335.766666667 130.166666667
second click
[(289, 116), (290, 116), (291, 116), (292, 116), (292, 117), (292, 118), (293, 118), (293, 119), (293, 120), (293, 121), (294, 121), (294, 122), (295, 122), (295, 123), (295, 124), (295, 125), (295, 126), (295, 127), (296, 127), (296, 128), (296, 129), (297, 129), (298, 129), (298, 130), (299, 130), (300, 130), (301, 130), (302, 130), (303, 130), (304, 130), (305, 130), (306, 130), (307, 130), (308, 130), (309, 130), (310, 130), (311, 130), (312, 130), (313, 130), (314, 130), (315, 130), (316, 130), (317, 130), (318, 130), (319, 130), (320, 130), (321, 130), (322, 130), (323, 130), (324, 130), (325, 130), (326, 130), (327, 130), (328, 130), (329, 130), (330, 130), (331, 130), (332, 130), (333, 130), (334, 130), (335, 130)]
306.875 87.7604166667
first click
354.270833333 116.927083333
second click
[(306, 87), (306, 86), (306, 85), (307, 85), (308, 85), (308, 86), (309, 86), (310, 86), (310, 87), (311, 87), (312, 87